##CNNを用いたmnist画像認識
mnistの手書き数字をCNNを用いて認識を行い、その結果を表示する。また作成したCNNについてはファイルとして出力する。

In [0]:
import chainer
import chainer.functions as F
import chainer.links as L
import numpy as np
from chainer import Chain, optimizers, Variable, serializers

In [3]:
from sklearn.datasets import fetch_mldata
from shutil import copyfileobj
from six.moves import urllib
from sklearn.datasets.base import get_data_home
import os

#mnist = fetch_mldata('MNIST original', data_home=".")
#このコードでエラーが出る場合は，以下を実行．

def fetch_mnist(data_home=None):
    mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
    data_home = get_data_home(data_home=data_home)
    data_home = os.path.join(data_home, 'mldata')
    if not os.path.exists(data_home):
        os.makedirs(data_home)
    mnist_save_path = os.path.join(data_home, "mnist-original.mat")
    if not os.path.exists(mnist_save_path):
        mnist_url = urllib.request.urlopen(mnist_alternative_url)
        with open(mnist_save_path, "wb") as matlab_file:
            copyfileobj(mnist_url, matlab_file)

fetch_mnist()
mnist = fetch_mldata('MNIST original')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
import matplotlib.pyplot as plot
#表示用メソッド
def draw_digit(data):
    size = 28
    plot.figure(figsize=(1, 1))

    X, Y = np.meshgrid(range(size),range(size))
    Z = data.reshape(size,size)   # convert from vector to 28x28 matrix
    Z = Z[::-1,:]             # flip vertical
    plot.xlim(0,27)
    plot.ylim(0,27)
    plot.pcolor(X, Y, Z)
    plot.gray()
    plot.tick_params(labelbottom="off")
    plot.tick_params(labelleft="off")

    plot.show()


In [5]:
x   = mnist.data.astype(np.float32)#入力値
x  /= 255     # 0-1のデータに変換
t = mnist.target.astype(np.int32)#目標値
from sklearn.model_selection import train_test_split
#データセットの分割
x_train_val, x_test, t_train_val, t_test = train_test_split(x, t, test_size=0.3, random_state=0)
x_train, x_val, t_train, t_val = train_test_split(x_train_val, t_train_val, test_size=0.3, random_state=0)

#データの整形
x_train = x_train.reshape((len(x_train), 1, 28, 28))
print(np.shape(x_train))
x_test = x_test.reshape((len(x_test), 1, 28, 28))
print(np.shape(t_train))

(34300, 1, 28, 28)
(34300,)


#convolution2Dについて
L.Convolution2D(inputPtn, convOutPtn, filtersize)   
inputPtn:入力画像が何次元か グレースケールなら1 カラーなら3  
convOutPtn:  
filtersize: フィルターのサイズ3×4なら(3,4)　5*5 なら5

In [0]:
from chainer import Sequential
# 入力 784次元、出力 10次元
# 中間層の数
#convolution2D()
n_units   = 1000
class CNN(Chain):
    def __init__(self):
        super(CNN, self).__init__(
            conv1 = L.Convolution2D(1, 20, 5), # filter 5
            conv2 = L.Convolution2D(20, 50, 5), # filter 5
            l1 = L.Linear(800, 500),
            l2 = L.Linear(500, 500),
            l3 = L.Linear(500, 10, initialW=np.zeros((10, 500), dtype=np.float32))
        )
    def forward(self, x):
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.relu(self.l1(h))
        h = F.relu(self.l2(h))
        h = self.l3(h)
        return h

In [7]:
#最適化手法
model = CNN()
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [8]:
n_epoch = 2
batch_size = 1000
for epoch in range(n_epoch):
    sum_loss = 0
    sum_accuracy = 0
    perm = np.random.permutation(len(x_train))
    for i in range(0, len(x_train), batch_size):
        x = Variable(x_train[perm[i:i+batch_size]])
        t = Variable(t_train[perm[i:i+batch_size]])
        y = model.forward(x)
        model.zerograds()
        loss = F.softmax_cross_entropy(y, t)
        acc = F.accuracy(y, t)
        loss.backward()
        optimizer.update()
        sum_loss += loss.data*batch_size
        sum_accuracy += acc.data*batch_size
    print("epoch: {}, mean loss: {}, mean accuracy: {}".format(epoch, sum_loss/len(x_train), sum_accuracy/len(x_train)))
    print(acc.data)
    print(type(acc.data))

#testに対する学習
cnt = 0
for i in range(10000):
    x = Variable(np.array([x_test[i]], dtype=np.float32))
    t = t_test[i]
    y = model.forward(x)
    y = np.argmax(y.data[0])
    if t == y:
        cnt += 1

print("accuracy: {}".format(cnt/10000))

epoch: 0, mean loss: 0.9788752714329488, mean accuracy: 0.742653059412022
0.9
<class 'numpy.ndarray'>
epoch: 1, mean loss: 0.24229349269811334, mean accuracy: 0.94772595010763
0.95
<class 'numpy.ndarray'>
accuracy: 0.9501


In [0]:
chainer.serializers.save_npz('num_cnn.net', model)

# 結果
NNよりはaccuracyいい感じ

## 参考文献
optimizerの最適化手法：  
MomentumSGDが一番いいらしい
https://www.iandprogram.net/entry/2016/02/11/181322#%E6%9C%80%E9%81%A9%E5%8C%96%E3%81%A8%E3%81%AF